In [1]:
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import load_model
import random
import json
import pickle

In [2]:
lemmatizer = WordNetLemmatizer()

with open('intents.json', 'r') as data_file:
    intents = json.load(data_file)
    
with open('words.pkl', 'rb') as word_file:
    words = pickle.load(word_file)

with open('classes.pkl', 'rb') as class_file:
    classes = pickle.load(class_file)

In [3]:
nltk.data.path.append('/home/chamath/Documents/PyProjects/ChatBot')

In [4]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

In [5]:
def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print("Found in bag: %s" % w)
    return  np.array(bag)              

In [7]:
model = load_model('chatbot_model2.h5')

In [8]:
def predict_class(sentence, model):
    p = bow(sentence, words, show_details=True)
    
    res = model.predict(np.array([p]))[0]
    
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    return return_list

In [9]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    
    for i in list_of_intents:
        if (i['tag'] == tag):
            result = random.choice(i['responses'])
            break
    return result

In [10]:
def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

In [11]:
import tkinter
from tkinter import *

In [12]:
def send():
    msg = EntryBox.get('1.0', 'end-1c').strip()
    EntryBox.delete('0.0', END)
    
    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, 'You: ' + msg + '\n\n')
        ChatLog.config(foreground='#442265', font=('Verdana', 12 ))
        
        res = chatbot_response(msg)
        ChatLog.insert(END, 'Bot: ' + res + '\n\n')
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)

In [13]:
base = Tk()
base.title('Hello')
base.geometry('400x500')
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg='white', height='8', width='50', font='Arial')

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor='heart')
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send)

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")

#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()

Found in bag: hi
Found in bag: what
Found in bag: you
Found in bag: can
Found in bag: do
Found in bag: i
Found in bag: want
Found in bag: to
Found in bag: thank
Found in bag: you
